This notebook is a driver program for _taxes_ library

In [12]:
%load_ext autoreload
%autoreload 2

from taxes.download import get_sheet_links_names, download_sheet_series
import pandas as pd
import pathlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# getting all video links 
year = 2019
sheets = get_sheet_links_names(year) 

dir_sheets = download_sheet_series(sheets)

print(sheets, dir_sheets)

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Gminy_za_2019.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Powiaty_za_2019.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Miasta_NPP_za_2019.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Wojewodztwa_za_2019.xlsx downloaded!

All sheets downloaded!
(['https://www.gov.pl/attachment/6594af94-cd1e-49fb-9149-99fd663aef25', 'https://www.gov.pl/attachment/5f2abc44-6a7e-4b73-8999-696920252efc', 'https://www.gov.pl/attachment/141da745-800d-44c5-ac97-e90c4cbd5e11', 'https://www.gov.pl/attachment/12150aff-d70e-

In [35]:
fl = pathlib.Path("/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data/20200214_Gminy_za_2019.xlsx")
col_names = ["WK", "PK", "GK", "GT", "Nazwa JST", "województwo", "powiat", "DZIAŁ", "ROZDZIAŁ", "PARAGRAF", 
             "Należności (saldo początkowe plus przypisy minus odpisy)", "Dochody wykonane (wpłaty minus zwroty)", 
             "ogółem", "zaległości netto", "nadpłaty"]
df = pd.read_excel(fl, skiprows=7, header=None, names=col_names)

In [57]:
# df_1 = df[df["Nazwa JST"] == "BOLESŁAWIEC"]
# df_1 = df_1[df_1["województwo"] == "dolnośląskie"]
# df_1

In [56]:
df.describe()

,WK,PK,GK,GT,DZIAŁ,ROZDZIAŁ,PARAGRAF,Należności (saldo początkowe plus przypisy minus odpisy),Dochody wykonane (wpłaty minus zwroty),ogółem,zaległości netto,nadpłaty
count,2411.000000,2411.000000,2411.000000,2411.000000,2411.0,2411.0,2411.0,2.411000e+03,2.411000e+03,2411.000000,2411.000000,0.0
mean,16.339278,11.294484,4.983409,2.167151,756.0,75621.0,10.0,9.639625e+06,9.638111e+06,1513.510991,1513.510991,NaN
std,9.178146,7.596322,3.250875,0.578895,0.0,0.0,0.0,1.344408e+07,1.344197e+07,2110.853530,2110.853530,NaN
min,2.000000,1.000000,1.000000,1.000000,756.0,75621.0,10.0,4.456260e+05,4.455560e+05,70.000000,70.000000,NaN
25%,10.000000,5.000000,2.000000,2.000000,756.0,75621.0,10.0,2.647580e+06,2.647164e+06,416.000000,416.000000,NaN
50%,14.000000,10.000000,4.000000,2.000000,756.0,75621.0,10.0,4.776931e+06,4.776181e+06,750.000000,750.000000,NaN
75%,24.000000,16.000000,7.000000,3.000000,756.0,75621.0,10.0,1.054709e+07,1.054544e+07,1656.000000,1656.000000,NaN
max,32.000000,38.000000,19.000000,3.000000,756.0,75621.0,10.0,2.157518e+08,2.157179e+08,33875.000000,33875.000000,NaN
